In [1]:
################################
##### Tokenization Process #####
################################

In [3]:
# Importing Necessary Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Importing Tokeniztion libraries
## keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

import warnings


# Going to import this random Tokenizer Module
import tokenize as tk

## word_idx = tokenizer.word_index


In [4]:
# Importing the Data Set

## This is the Original Federer vs Roddick Dataset
fr = pd.read_csv('FedRod.csv')

### This is the dataset where only Andy Roddick is Serving
ar_serve = fr[(fr['Serving']== 'AR')]

#### This is the dataset where Andy Roddick is only Serving to the Duece Side
ar_d_srv = ar_serve[(ar_serve['SrvDueceSide'])==True]

##### This is the dataset which only uses points where the First Serve Went In
ar_token = ar_d_srv[(ar_d_srv['Sv2'].isnull())]

###### Let's Double Check
ar_token.head()

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,SrvDueceSide,Gm#,TbSet,...,PtsAfter,GmW,Gm1.1,Gm2.1,SetW,Set1.1,Set2.1,RevTB,TBrev,rallyCount
7,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,8,0,0,1,0,15-15,True,2,1,...,30-15,0,1,0,0,0,0,NaN,NaN,1
9,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,10,0,0,1,0,30-30,True,2,1,...,40-30,0,1,0,0,0,0,NaN,NaN,7
11,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,12,0,0,1,0,40-40,True,2,1,...,AD-40,0,1,0,0,0,0,NaN,NaN,5
19,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,20,0,0,2,1,30-0,True,4,1,...,40-0,0,2,1,0,0,0,NaN,NaN,3
26,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,27,0,0,3,2,0-0,True,6,1,...,15-0,0,3,2,0,0,0,NaN,NaN,1


In [5]:
# Let's simplify this process even further by trying to split just one string

## In this String, Roddick hit's a first serve in and loses an 11 Shot Rally
string = '5f38f3b3b3b2f1f1f3s3b2b1*'

### We want to split the string
string = string.replace("f", " f")
string = string.replace("b", " b")
string = string.replace("r", " r")
string = string.replace("s", " s")
string = string.replace("v", " v")
string = string.replace("z", " z")
string = string.replace("o", " o")
string = string.replace("p", " p")
string = string.replace("l", " l")
string = string.replace("m", " m")
string = string.replace("h", " h")
string = string.replace("i", " i")
string = string.replace("j", " j")
string = string.replace("k", " k")
string = string.replace("t", " t")
string = string.replace("q", " q")


print(string)


5 f38 f3 b3 b3 b2 f1 f1 f3 s3 b2 b1*


In [6]:
# Below we will create a Function that splits a Rally Sequence by each Shot

def token_maker(string):
    string = string.replace("f", " f")
    string = string.replace("b", " b")
    string = string.replace("r", " r")
    string = string.replace("s", " s")
    string = string.replace("v", " v")
    string = string.replace("z", " z")
    string = string.replace("o", " o")
    string = string.replace("p", " p")
    string = string.replace("l", " l")
    string = string.replace("m", " m")
    string = string.replace("h", " h")
    string = string.replace("i", " i")
    string = string.replace("j", " j")
    string = string.replace("k", " k")
    string = string.replace("t", " t")
    string = string.replace("q", " q")
    
    return print(string)

In [7]:
# Here we are creating a Test Dataset for Testing our Token Creating Function

tester = pd.read_csv('TestToken.csv')

c = 0

while c < len(tester.index):
    token_maker(tester.iloc[c,15])
    c=c+1

6 s27 f+1 f2 v#
4w
6n
6 f2d#
4 f28 f3 s3 f1 f3 s3 b3 b2 f3w@
6n
6*
5 s2d#
5 f38 f3 b3 b3 b2 f1 f1 f3 s3 b2 b1*
5 s28 f+3 b2 z3 b2 v2 r3x#


In [16]:
# This is going to be pretty ugly, but, I'm going to:
# Create a CSV of the Roddick Duece Side 1st Sv In Data ONLY
# and then run the function against that data, then copy paste the output
# into a new row (I know this can be done in python I'm just completely
# blanking on it at the moment). This will give us tokenized data at least!

ar_token.to_csv(r"C:\Users\nicho\Desktop\Udemy Python\Machine Learning (UTS)\Assignment 2\ar_token.csv")

# (This can be ignored for now)

In [8]:
# Below I am just testing the function above for formatting the data so that it can be tokenized

tester = pd.read_csv('ar_token.csv')

c = 0

while c < len(tester.index):
    token_maker(tester.iloc[c,16])
    c=c+1

5 s2d#
5 s28 f+3 b2 z3 b2 v2 r3x#
4 f27 f+3 b1 v1*
5 s29 f1 f1n@
4+ f#
4+*
5 b2n#
6*
6*
6*
6 s28 f1 f3n@
5 s#
4 f2d#
4 f2n#
4 f2d#
6 s27 f-3*
4 f38 b2 b-1*
4*
4 f#
6 s#
5 b3w#
6 b#
5 s2n#
4 f28 f+3 s1n#
4 r27 f+1 f1 v3 s1d#
6 s38 b3 s3 b1 f1d@
4*
6 s#
4 f39 b2 f1 f2 f2 f+3 s2 v3*
4 r2n#
4 f28 f+3 b2 z1 l2 o3*
4 f3w#
5 s28 f+3*
c4 f28 s3 b1n@
6*
6*
5 s27 f+2 b2 z2 f1 v1w#
4 f18 f3w#
5 b;27 f-2*
6 b3w#
6 b2d#
6 b38 f3 b1*
4 f1d#
5 f28 f2 f2 f3 b3 b3 s1 f1 f3*
4 f2n#
4 f27 f3*
4+ f38*
4 f27 f+-3 b1d#
6*
6*
5 b18 f1 f1n#
4+ f17 f-2 i=3*
6 b28 f1 f1 v+-3 b1*
4 f29 f2 s2 f1 f1 f3d#
4 f29 b3 b1 f3+ b3*
4 f29 f3 b3 f+-1 l2 o3*
5 s28 f1 f1 f3+ s1*
4 f1d#
4 f29*
6 b27 f+-2 f2 z-3*
c4 f28 f3+ b3*
c4 f29 b2 f+-3*
4 f1w#
c6 s27 f+3d@
4 f29 f+3 b1*
4 f28 b+3 b3 z1 f1*
4#
4#
4+ f v3n@
4 r29 f+3 b3 z1 l2d#
4 f28 f+1 l#
6 b38 b+3 b3 z3 m#
4#
4 f39 b3 b2 f3 b2 f3 b3n@
4 f29 s3 b3 b3 b1 f1 f+3 b1*
6#
4 f19 f2 s3 b3 b3 f3 b1*
4 r29 f+3 b3 z1*
6#
4 f18 f+3 b3 z1 l3 o3 m1 f2 o3*
4 f19 f+3 b1n#
4*
6#
4#
6*
6

In [9]:
# So below is our now Formatted AR Serving to the FH side 
t_ar = pd.read_csv('t_ar.csv')
t_ar.head()

,Unnamed: 0,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,SrvDueceSide,Gm#,...,PtsAfter,GmW,Gm1.1,Gm2.1,SetW,Set1.1,Set2.1,RevTB,TBrev,rallyCount
0,7,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,8,0,0,1,0,15-15,True,2,...,30-15,0,1,0,0,0,0,NaN,NaN,1
1,9,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,10,0,0,1,0,30-30,True,2,...,40-30,0,1,0,0,0,0,NaN,NaN,7
2,11,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,12,0,0,1,0,40-40,True,2,...,AD-40,0,1,0,0,0,0,NaN,NaN,5
3,19,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,20,0,0,2,1,30-0,True,4,...,40-0,0,2,1,0,0,0,NaN,NaN,3
4,26,20070905-M-US_Open-QF-Roger_Federer-Andy_Roddick,27,0,0,3,2,0-0,True,6,...,15-0,0,3,2,0,0,0,NaN,NaN,1


In [10]:
# OK now we're going to try Count all of the strings in the Tokenized COlumn




# Turning t_ar[1st] into a list

list_t_ar = t_ar['1st'].tolist()


## This was so that we could tokenize the list
## Now we're going to use the tokenizer module
## "Filters ='.'" was used so that I don't lose values such as @ or # [values important for point endings]

tokenizer = Tokenizer(num_words=None, filters='.', lower=True, split = ' ')

### Train our tokenziser to the strings (PLEASE WORK!)
tokenizer.fit_on_texts(list_t_ar)

#### Convert our list of strings into a list of integers
sequences = tokenizer.texts_to_sequences(list_t_ar)

##### now all of our shots in rally are represented as numbers instead of code
##### We now need to map tokenized numbers to values
##### Mapping Indexes to Words
idx_word = tokenizer.index_word

' '.join(idx_word[w] for w in sequences[100][:40])




'4 f39 s2 f1 f2 s2 b1n@'

In [18]:
# Just for Further proof ^^^ This is the same as above

sequences[100][:40]

[1, 26, 19, 2, 8, 19, 48]

In [23]:
# We've "Tokenized" our data, now we're going give our network a sequence of words, and train the model to predict the next word

features = []
labels = []

training_length = 50

## This is the training length used in the example I learnt from
## It means we're going to train a sequence with 50 tokens (this seems was too long)
## But that's ok we're simply testing at the moment

### Now we need to iterate through our sequence of tokens
for seq in sequences:
    
    #### We need to create multiple training examples from each sequence
    for i in range(training_length, len(seq)):
        
        ##### We need to now extract the features and label
        extract = seq[i - training_length:i + 1]
        
        ##### We need to now Set the features and the label
        features.append(extract[:-1])
        labels.append(extract[-1])
        
features = np.array(features)

In [24]:
features.shape

(0,)

In [21]:
# In order to train the neural network, we need to one-hot encode the labels
# This is essentially binarising, so if we had the category "Colours" with categories "Red" "Blue" " Green:
# Our Table would become
##############################
### Red ### Blue ### Green ###
##------###------###-------###
###  0  ###   1  ###   0   ###
##------###------###-------###
###  1  ###   0  ###   0   ###
##------###------###-------###



## To do this:
num_words = len(word_idx) + 1

## Empty the array to hold labels
label_array = np.zeros((len(features), num_words), dtype = np.int8)

## One hot-encode the labels
for example_index, word_index in enumerate(labels):
    label_array[example_index,word_index] = 1
    
label_array.shape

NameError: name 'word_idx' is not defined